In [14]:
import sys
sys.path.append('D:\PyCharmProjects\VFPUMC02')
sys.path.append(r'C:\Users\Administrator\PycharmProjects\VFPUMC02')
sys.path.append(r'/root/VFPUMC02')
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import *
from consts.Constants import DATASETS_PATH
import os
from datasets.VFPU_GEN_DATASET import VFPU_GEN_DATASET
from enums.DataSetsName import DataSetsName
from ctgan import CTGAN
from classfiers.TwoStep import TwoStep
import pandas as pd
from vf4lr.VF_LR import VF_LR
from classfiers.VF_LinearRegression import VF_LinearRegression
from classfiers.VF_TwoStep import VF_TwoStep
from utils.FateUtils import convert_ipynb_to_py
from gen.VFPU_GEN import VFPU_GEN
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

In [16]:
dataset = VFPU_GEN_DATASET()
df_A, df_B, y = dataset.get_dataset_parts(DataSetsName.BANK.value)
miss_rate=0.2
complete_df_B, incomplete_df_B, missing_df_B = constru_row_miss_df(df_B, miss_rate)

In [17]:
ctgan = CTGAN(epochs=5)

clf =  LogisticRegression(solver='sag', max_iter=5)
reg =  Ridge(alpha=0.1, solver='sag', max_iter=5)
two_step = TwoStep(
    base_classifier=clf,
    base_regressor=reg,
    max_iter=5,
    k=0.25
)

vf_clf = VF_LR(learning_rate=0.2,epoch_num=5,batch_size=64)
vf_reg = VF_LinearRegression(config={'lr': 0.01,'lambda': 0.1,'n_iters':5})
vf_two_step = VF_TwoStep(
    clf=vf_clf,
    reg=vf_reg,
    k=0.25,
    max_iter=5
)

2025-01-23 23:05:11,567 - ConsoleLogger - INFO - TwoStep 类已初始化: max_iter=5, k=0.25, base_classifier=LogisticRegression, base_regressor=Ridge
2025-01-23 23:05:11,567 - INFO - TwoStep 类已初始化: max_iter=5, k=0.25, base_classifier=LogisticRegression, base_regressor=Ridge
2025-01-23 23:05:11,568 - ConsoleLogger - INFO - VF_LR 模型初始化完成。
2025-01-23 23:05:11,568 - INFO - VF_LR 模型初始化完成。
2025-01-23 23:05:11,569 - ConsoleLogger - INFO - 模型配置: {'learning_rate': 0.2, 'epoch_num': 5, 'batch_size': 64, 'client_num': 2}
2025-01-23 23:05:11,569 - INFO - 模型配置: {'learning_rate': 0.2, 'epoch_num': 5, 'batch_size': 64, 'client_num': 2}
2025-01-23 23:05:11,570 - ConsoleLogger - INFO - VF_LinearRegression 模型初始化完成。
2025-01-23 23:05:11,570 - INFO - VF_LinearRegression 模型初始化完成。
2025-01-23 23:05:11,570 - ConsoleLogger - INFO - 模型配置: {'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}
2025-01-23 23:05:11,570 - INFO - 模型配置: {'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}
2025-01-23 23:05:11,571 - INFO - VF_TwoStep 类已成功初始化: max_iter=

In [18]:
vfpu_gen = VFPU_GEN(two_step,vf_two_step,ctgan,df_A,incomplete_df_B)

In [19]:
vfpu_gen.fit(predict_cols=None)
syn_data_gen_all = vfpu_gen.get_synthetic_data()

训练完成


In [20]:
vfpu_gen.fit(incomplete_df_B.columns)
syn_data_predict_all = vfpu_gen.get_synthetic_data()

2025-01-23 23:05:36,182 - ConsoleLogger - INFO - 有标签数据 (L): 32951 样本
2025-01-23 23:05:36,182 - INFO - 有标签数据 (L): 32951 样本
2025-01-23 23:05:36,184 - ConsoleLogger - INFO - 无标签数据 (U): 8237 样本
2025-01-23 23:05:36,184 - INFO - 无标签数据 (U): 8237 样本
2025-01-23 23:05:36,184 - ConsoleLogger - INFO - 任务类型判定结果: classification
2025-01-23 23:05:36,184 - INFO - 任务类型判定结果: classification
2025-01-23 23:05:36,185 - ConsoleLogger - INFO - 这是一个分类任务。
2025-01-23 23:05:36,185 - INFO - 这是一个分类任务。
2025-01-23 23:05:36,187 - ConsoleLogger - INFO - 分类任务中的类别: [0 1]
2025-01-23 23:05:36,187 - INFO - 分类任务中的类别: [0 1]
2025-01-23 23:05:36,189 - ConsoleLogger - INFO - 初始化预测结果数组，长度=8237
2025-01-23 23:05:36,189 - INFO - 初始化预测结果数组，长度=8237
2025-01-23 23:05:36,190 - ConsoleLogger - INFO - 开始自训练迭代，共计划进行 5 次迭代
2025-01-23 23:05:36,190 - INFO - 开始自训练迭代，共计划进行 5 次迭代
2025-01-23 23:05:36,190 - ConsoleLogger - INFO - ===== 第 0 次迭代开始 =====
2025-01-23 23:05:36,190 - INFO - ===== 第 0 次迭代开始 =====
2025-01-23 23:05:36,191 - ConsoleLogger - IN

=== Logs ===
df_A_L: (32951, 31)
df_A_U: (8237, 31)
construct_df_B_L: (32951, 32)
construct_df_B_U: (8237, 32)
train_cols: []
y_L_dict keys: ['job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'loan_no', 'loan_unknown', 'loan_yes', 'marital_divorced', 'marital_married', 'marital_single', 'marital_unknown', 'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep', 'nr.employed', 'pdays', 'poutcome_failure', 'poutcome_nonexistent', 'poutcome_success', 'previous']
y_U_dict keys: ['job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'loan_no', 'loan_unknown', 'loan_yes', 'marital_divorced', 'marital_married', 'marital_single', 'marital_unknown', 'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar'

/root/miniconda3/envs/fate_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
2025-01-23 23:05:36,491 - ConsoleLogger - INFO - 分类器训练完成，耗时 0.15 秒。
2025-01-23 23:05:36,491 - INFO - 分类器训练完成，耗时 0.15 秒。
2025-01-23 23:05:36,492 - ConsoleLogger - INFO - 分类器识别的类别: [0 1]
2025-01-23 23:05:36,492 - INFO - 分类器识别的类别: [0 1]
2025-01-23 23:05:36,493 - ConsoleLogger - INFO - 开始对 Unlabeled 数据进行置信度打分...
2025-01-23 23:05:36,493 - INFO - 开始对 Unlabeled 数据进行置信度打分...
2025-01-23 23:05:36,495 - ConsoleLogger - INFO - 完成置信度打分，耗时 0.00 秒。
2025-01-23 23:05:36,495 - INFO - 完成置信度打分，耗时 0.00 秒。
2025-01-23 23:05:36,497 - ConsoleLogger - INFO - 选取前 25.00% 的高置信度样本。
2025-01-23 23:05:36,497 - INFO - 选取前 25.00% 的高置信度样本。
2025-01-23 23:05:36,498 - ConsoleLogger - INFO - 本轮选出高置信度样本数量: 1544，占比约为 25.00%
2025-01-23 23:05:36,498 - INFO - 本轮选出高置信度样本数量: 1544，占比约为 25.00%
2025-01-23 23:05:36,499 - ConsoleLogger - INFO - 

训练完成


In [21]:
syn_data_predict_all.shape,syn_data_gen_all.shape, missing_df_B.shape

((8237, 32), (8237, 32), (8237, 32))

In [22]:
vfpu_gen.fit(predict_cols=['job_housemaid'])
syn_data_predict_1 = vfpu_gen.get_synthetic_data()

2025-01-23 23:06:04,311 - INFO - 有标签数据 (L): 32951 样本
2025-01-23 23:06:04,312 - INFO - 无标签数据 (U): 8237 样本
2025-01-23 23:06:04,312 - INFO - 任务类型判定结果: classification
2025-01-23 23:06:04,313 - INFO - 这是一个分类任务。
2025-01-23 23:06:04,315 - INFO - 分类任务中的类别: [0 1]
2025-01-23 23:06:04,316 - INFO - [INIT] 初始化 self.pred, 未标注数据数量=8237
2025-01-23 23:06:04,317 - INFO - [INIT] 最大迭代次数 max_iter=5, 每轮选取比例 k=0.25
2025-01-23 23:06:04,318 - INFO - ===== [Epoch 1/5] 迭代开始 =====
2025-01-23 23:06:04,318 - INFO - [INFO] 当前 Labeled 样本数量: 32951, Unlabeled 样本数量: 8237
2025-01-23 23:06:04,319 - INFO - [STEP 1] 开始训练分类器 (基于 32951 个 Labeled 样本)...
2025-01-23 23:06:04,321 - ConsoleLogger - INFO - 开始训练垂直联邦逻辑回归模型，客户端A训练数据形状: (32951, 31), 客户端B训练数据形状: (32951, 31), 标签大小: 32951
2025-01-23 23:06:04,321 - INFO - 开始训练垂直联邦逻辑回归模型，客户端A训练数据形状: (32951, 31), 客户端B训练数据形状: (32951, 31), 标签大小: 32951
2025-01-23 23:06:04,322 - ConsoleLogger - INFO - 检测到的类别数量: 2
2025-01-23 23:06:04,322 - INFO - 检测到的类别数量: 2
2025-01-23 23:06:04,323 - ConsoleLogge

=== Logs ===
df_A_L: (32951, 31)
df_A_U: (8237, 31)
construct_df_B_L: (32951, 32)
construct_df_B_U: (8237, 32)
train_cols: ['job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'loan_no', 'loan_unknown', 'loan_yes', 'marital_divorced', 'marital_married', 'marital_single', 'marital_unknown', 'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep', 'nr.employed', 'pdays', 'poutcome_failure', 'poutcome_nonexistent', 'poutcome_success', 'previous']
y_L_dict keys: ['job_housemaid']
y_U_dict keys: ['job_housemaid']
construct_df_B_L_train: (32951, 31)
construct_df_B_U_train: (8237, 31)
=== End of Logs ===

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-23 23:06:04,980 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 23:06:04,980 - INFO - 第 1 轮训练完成。
2025-01-23 23:06:04,981 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:04,981 - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:05,598 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 23:06:05,598 - INFO - 第 2 轮训练完成。
2025-01-23 23:06:05,600 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:05,600 - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:06,227 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 23:06:06,227 - INFO - 第 3 轮训练完成。
2025-01-23 23:06:06,229 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:06,229 - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:06,843 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 23:06:06,843 - INFO - 第 4 轮训练完成。
2025-01-23 23:06:06,845 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:06,845 - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:07,509 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 23:06:07,509 - INFO - 第 5 轮训练完成。
2025-01-23 23:06:07,510 - Consol

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-23 23:06:11,424 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 23:06:11,424 - INFO - 第 1 轮训练完成。
2025-01-23 23:06:11,426 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:11,426 - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:11,908 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 23:06:11,908 - INFO - 第 2 轮训练完成。
2025-01-23 23:06:11,909 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:11,909 - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:12,385 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 23:06:12,385 - INFO - 第 3 轮训练完成。
2025-01-23 23:06:12,386 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:12,386 - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:12,879 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 23:06:12,879 - INFO - 第 4 轮训练完成。
2025-01-23 23:06:12,880 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:12,880 - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:13,382 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 23:06:13,382 - INFO - 第 5 轮训练完成。
2025-01-23 23:06:13,383 - Consol

...

2025-01-23 23:06:16,642 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 23:06:16,642 - INFO - 第 1 轮训练完成。
2025-01-23 23:06:16,643 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:16,643 - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:17,142 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 23:06:17,142 - INFO - 第 2 轮训练完成。
2025-01-23 23:06:17,143 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:17,143 - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:17,666 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 23:06:17,666 - INFO - 第 3 轮训练完成。
2025-01-23 23:06:17,667 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:17,667 - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:18,181 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 23:06:18,181 - INFO - 第 4 轮训练完成。
2025-01-23 23:06:18,182 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:18,182 - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:18,700 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 23:06:18,700 - INFO - 第 5 轮训练完成。
2025-01-23 23:06:18,704 - Consol

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-23 23:06:22,205 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 23:06:22,205 - INFO - 第 1 轮训练完成。
2025-01-23 23:06:22,206 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:22,206 - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:22,758 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 23:06:22,758 - INFO - 第 2 轮训练完成。
2025-01-23 23:06:22,760 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:22,760 - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:23,291 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 23:06:23,291 - INFO - 第 3 轮训练完成。
2025-01-23 23:06:23,293 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:23,293 - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:23,840 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 23:06:23,840 - INFO - 第 4 轮训练完成。
2025-01-23 23:06:23,841 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:23,841 - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:24,368 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 23:06:24,368 - INFO - 第 5 轮训练完成。
2025-01-23 23:06:24,369 - Consol

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-23 23:06:27,771 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 23:06:27,771 - INFO - 第 1 轮训练完成。
2025-01-23 23:06:27,772 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:27,772 - INFO - 开始第 2/5 轮训练...
2025-01-23 23:06:28,296 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 23:06:28,296 - INFO - 第 2 轮训练完成。
2025-01-23 23:06:28,297 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:28,297 - INFO - 开始第 3/5 轮训练...
2025-01-23 23:06:28,831 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 23:06:28,831 - INFO - 第 3 轮训练完成。
2025-01-23 23:06:28,832 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:28,832 - INFO - 开始第 4/5 轮训练...
2025-01-23 23:06:29,365 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 23:06:29,365 - INFO - 第 4 轮训练完成。
2025-01-23 23:06:29,366 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:29,366 - INFO - 开始第 5/5 轮训练...
2025-01-23 23:06:29,911 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 23:06:29,911 - INFO - 第 5 轮训练完成。
2025-01-23 23:06:29,913 - Consol

In [63]:
syn_data_gen_all

,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,loan_no,...,month_may,month_nov,month_oct,month_sep,nr.employed,pdays,poutcome_failure,poutcome_nonexistent,poutcome_success,previous
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-0.940281,0.195414,0,0,0,-0.349494
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0.845170,0.195414,0,1,0,-0.349494
2,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0.845170,0.195414,0,1,1,-0.349494
3,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0.845170,0.195414,0,1,0,-0.349494
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0.845170,0.195414,0,1,0,-0.349494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8232,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0.845170,0.195414,0,1,0,-0.349494
8233,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0.845170,0.195414,0,0,0,-0.349494
8234,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0.398115,0.195414,0,0,1,-0.349494
8235,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0.331680,0.195414,0,1,0,1.671136


In [67]:
get_identical_columns(syn_data_gen_all, syn_data_predict_1)

['job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_unknown',
 'loan_no',
 'loan_unknown',
 'loan_yes',
 'marital_divorced',
 'marital_married',
 'marital_single',
 'marital_unknown',
 'month_apr',
 'month_aug',
 'month_dec',
 'month_jul',
 'month_jun',
 'month_mar',
 'month_may',
 'month_nov',
 'month_oct',
 'month_sep',
 'nr.employed',
 'pdays',
 'poutcome_failure',
 'poutcome_nonexistent',
 'poutcome_success',
 'previous']

In [70]:
compare_dataframe_columns(syn_data_gen_all, syn_data_predict_1)

(['job_management',
  'job_retired',
  'job_self-employed',
  'job_services',
  'job_student',
  'job_technician',
  'job_unemployed',
  'job_unknown',
  'loan_no',
  'loan_unknown',
  'loan_yes',
  'marital_divorced',
  'marital_married',
  'marital_single',
  'marital_unknown',
  'month_apr',
  'month_aug',
  'month_dec',
  'month_jul',
  'month_jun',
  'month_mar',
  'month_may',
  'month_nov',
  'month_oct',
  'month_sep',
  'nr.employed',
  'pdays',
  'poutcome_failure',
  'poutcome_nonexistent',
  'poutcome_success',
  'previous'],
 ['job_housemaid'])